In [2]:
import os
import torch

from fairseq.models.bart import BARTModel
#from rouge import Rouge, FilesRouge
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("gpu num: ", n_gpu)

def create_hypotheses(weights_path, data_path, source, source2):
    bart = BARTModel.from_pretrained(
        weights_path,
        checkpoint_file='checkpoint_best.pt',
        data_name_or_path=data_path
    )

    bart.eval()
    bart.cuda()
    slines = [source.strip()]
    slines2 = [source2.strip()]
    hypotheses = bart.sample(slines, sentences2 = slines2, balance = True, beam=4, lenpen=2.0, max_len_b=100, min_len=5, no_repeat_ngram_size=3)
    torch.cuda.empty_cache()
    
    return hypotheses


def show_n_labels_and_preds(hyp_path, n=2):
    ref_path = './data/dialogsum/DialogSum_Data/test_dialogsum_sent_trans_cons_label_2.target'
    hypothesis = []
    with open(hyp_path, 'r') as f:
            lines = f.readlines()
            for l in lines:
                hypothesis.append(l[:-1])

    reference = []
    with open(ref_path, 'r') as f:
        lines = f.readlines()
        for l in lines:
            reference.append(l[:-1])

    for i in range(n):
        print('PRED:')
        print(hypothesis[i])
        print('LABEL:')
        print(reference[i])
        print('\n')

gpu num:  1


In [13]:
pretrained_test_hypo = './data/dialogsum/DialogSum_Data/test_best_multi_attn_best_PRETRAINED.hypo'
show_n_labels_and_preds(pretrained_test_hypo)

PRED:
Person1# sends an intra-office memorandum to all employees. The use of Instant Message programs by employees during working hours is strictly prohibited. Any employee who persists in using Instant Messaging will first receive a warning and be placed on probation. At second offense, the employee will face termination. The policy applies to internal and external communications.
LABEL:
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.


PRED:
Person2# got stuck in a traffic jam near the Carrefour intersection. She will consider taking public transport system to work instead of driving to work.
LABEL:
#Person2# arrives late because of traffic jam. #Person1# persuades #Person2# to use public transportations to keep healthy and to protect the environment.




In [14]:
test_hypo_single = './data/dialogsum/DialogSum_Data/test_best_multi_attn_best_SINGLE.hypo'
show_n_labels_and_preds(test_hypo_single)

PRED:
#Person1# asks Ms. Dawson to take a dictation for him and asks her to send an intra-office memorandum to all employees by this afternoon. He tells her that Instant Message programs by employees during working hours are strictly prohibited and applies to internal and external communications. Any employee who persists in using Instant Messaging will first receive a warning and be placed on probation, and at second offense, the employee will face termination.
LABEL:
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.


PRED:
#Person2# got stuck in traffic again. #Person1# suggests taking public transport system to work to get home and suggests biking to work when it's nicer outside, but #Person2 # is going to miss having the freedom with a car. Then they talk about the environment.
LABEL:
#Person2# arrives late because of traffic jam. #Person1# persuades #Person2# to use p

In [15]:
test_hypo_multi = './data/dialogsum/DialogSum_Data/test_best_multi_attn_best_MULTI.hypo'
show_n_labels_and_preds(test_hypo_multi)

PRED:
#Person1# asks Ms. Dawson to take a dictation for the intra-office memorandum and tells the staff that all office communications are restricted to email correspondence and official memos and the use of Instant Message programs by employees during working hours is strictly prohibited. Any employee who persists in using Instant Messaging will first receive a warning and be placed on probation, and at second offense, the employee will face termination. #Person2# asks for clarification on the new policy.
LABEL:
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.


PRED:
#Person2# got stuck in traffic again. #Person1# suggests taking public transport system to work and suggests taking the subway, but #Person2 # thinks it's not good for the environment and the freedom with a car. Then they talk about biking to work when it's nicer outside.
LABEL:
#Person2# arrives late becaus

In [16]:
dialogue = "#Person1#: She said yes! #Person2#: What? #Person1#: I proposed to Sarah and she said yes! #Person2#: Oh my god! That's amazing! Let me visit you and you have to tell me everything! #Person1#: Yes, sure! | \n"
print(dialogue)

#Person1#: She said yes! #Person2#: What? #Person1#: I proposed to Sarah and she said yes! #Person2#: Oh my god! That's amazing! Let me visit you and you have to tell me everything! #Person1#: Yes, sure! | 



In [17]:
weights_path = 'checkpoints_multi_view'
data_path = './cnn_dm-bin_2'
hypotheses = create_hypotheses(weights_path, data_path, dialogue, dialogue)

here!


In [18]:
hypotheses

['#Person1 proposed to Sarah and she said yes. #Person2 will visit him.']

In [19]:
weights_path = 'train_sh/checkpoints_stage'
data_path = '../dialogsum-bin'
hypotheses = create_hypotheses(weights_path, data_path, dialogue, dialogue)
print(hypotheses[0])

here!
#Person1# proposed to Sarah and she said yes. #Person2#'s amazed.


In [20]:
weights_path = 'train_sh/checkpoints'
data_path = '../dialogsum-bin_2'
hypotheses = create_hypotheses(weights_path, data_path, dialogue, dialogue)
print(hypotheses[0])

here!
#Person1# proposes to Sarah and she said yes. #Person2# wants to visit.


In [13]:
dialogue = "#Person1#: Hello Mrs. Parker, how have you been?\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.\n"

In [14]:
for utt in dialogue.split('\n'):
    print(utt)

#Person1#: Hello Mrs. Parker, how have you been?
#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.
#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.
#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.



In [12]:
weights_path = 'checkpoints_multi_view'
data_path = './cnn_dm-bin_2'
create_hypotheses(weights_path, data_path, dialogue, dialogue)

here!


["Mrs. Parker's son, Ricky, has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old and is due for Hep-A, Chickenpox and Measles shots. The nurse will administer the booster to him."]

In [17]:
dialogue_2 = '#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person2#: What about Rubella and Mumps?\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n'
for utt in dialogue_2.split('\n'):
    print(utt)

#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.
#Person2#: What about Rubella and Mumps?
#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!



In [18]:
weights_path = 'checkpoints_multi_view'
data_path = './cnn_dm-bin_2'
create_hypotheses(weights_path, data_path, dialogue_2, dialogue_2)

here!


["#Person2# and Ricky are here for Ricky's vaccines. Ricky will get Rubella and Mumps, and he may need a Tetanus booster."]

In [15]:
dialogue = "#Person1#: Excuse me, did you see a set of keys?\n#Person1#: Five keys and a small foot ornament.\n#Person1#: Well, can you help me look for it? That's my first time here.\n#Person1#: It's very kind of you.\n#Person1#: Oh, thank God! I don't know how to thank you, guys.\n"
for utt in dialogue.split('\n'):
    print(utt)

#Person1#: Excuse me, did you see a set of keys?
#Person1#: Five keys and a small foot ornament.
#Person1#: Well, can you help me look for it? That's my first time here.
#Person1#: It's very kind of you.
#Person1#: Oh, thank God! I don't know how to thank you, guys.



In [16]:
weights_path = 'checkpoints_multi_view'
data_path = './cnn_dm-bin_2'
create_hypotheses(weights_path, data_path, dialogue, dialogue)

here!


['#Person1# is looking for a set of keys and a small foot ornament. He found it.']

In [19]:
dialogue_2 = "#Person2#: What kind of keys?\n#Person2#: What a shame! I didn't see them.\n#Person2#: Sure. It's my pleasure. I'd like to help you look for the missing keys.\n#Person2#: It's not a big deal.Hey, I found them.\n#Person2#: You're welcome.\n"
for utt in dialogue_2.split('\n'):
    print(utt)

#Person2#: What kind of keys?
#Person2#: What a shame! I didn't see them.
#Person2#: Sure. It's my pleasure. I'd like to help you look for the missing keys.
#Person2#: It's not a big deal.Hey, I found them.
#Person2#: You're welcome.



In [21]:
weights_path = 'checkpoints_multi_view'
data_path = './cnn_dm-bin_2'
create_hypotheses(weights_path, data_path, dialogue_2, dialogue_2)

here!


['#Person2# will look for the missing keys.']